In [ ]:
import pandas as pd
import csv
from pytrends.request import TrendReq

In [ ]:
pytrend = TrendReq()

In [ ]:
with open('geoInfo.csv') as csv_file:
    reader = csv.reader(csv_file)
    geo_dict = dict(reader)

In [ ]:
Americas = eval(geo_dict['Americas'])
EMEA = eval(geo_dict['EMEA'])
APAC = eval(geo_dict['APAC'])

In [ ]:
kw_list = ['Sony DSLR', 'Canon DSLR', 'Nikon DSLR']
pytrend.build_payload(kw_list, timeframe='2020-01-01 2020-03-31', geo='CA, US')

In [ ]:

# Interest by Region
#df = pytrends.interest_by_region()
df = pytrend.get_historical_interest(kw_list, year_start=2006, month_start=1, day_start=1, hour_start=0, \
                                      year_end=2020, month_end=4, day_end=1, hour_end=0, \
                                      cat=0, geo='', gprop='', sleep=30)
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.tail()